在 LangChain 框架中,并行执行由 LangChain Expression Language(LCEL)实现。主要方法是将多个 runnable 组件结构化为字典或列表,当这些集合被传递给链中的下一个组件时,LCEL 运行时会并发执行其中的 runnable。

在 LangGraph 中,这一原理体现在图的拓扑结构。通过设计图结构,使多个无直接依赖的节点可由同一节点并发启动,这些并行路径独立执行,结果在后续汇聚节点整合。

In [6]:
import os
import asyncio
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable, RunnableParallel, RunnablePassthrough

llm = ChatOpenAI(
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("BASE_URL"),
    model="deepseek-chat",
    temperature=0,
)

# ‐‐‐ 定义独立链 ‐‐‐
# 三个链分别执行不同任务,可并行运行
summarize_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "请简明扼要地总结以下主题:"),
            ("user", "{topic}"),
        ]
    )
    | llm
    | StrOutputParser()
)

question_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "请针对以下主题生成三个有趣的问题:"),
            ("user", "{topic}"),
        ]
    )
    | llm
    | StrOutputParser()
)

terms_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "请从以下主题中提取 5-10 个关键词,用逗号分隔:"),
            ("user", "{topic}"),
        ]
    )
    | llm
    | StrOutputParser()
)

# ‐‐‐ 构建并行 + 汇总链 ‐‐‐

# 1. 定义并行任务块,结果与原始 topic 一起传递到下一步
map_chain = RunnableParallel(
    {
        "summary": summarize_chain,
        "questions": question_chain,
        "key_terms": terms_chain,
        "topic": RunnablePassthrough(),  # 传递原始 topic
    }
)

# 2. 定义最终汇总 prompt,整合并行结果
synthesis_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """根据以下信息:
            摘要:{summary}
            相关问题:{questions}
            关键词:{key_terms}
            请综合生成完整答案。""",
        ),
        ("user", "原始主题:{topic}"),
    ]
)

# 3. 构建完整链,将并行结果直接传递给汇总 prompt,再由 LLM 和输出解析器处理
full_parallel_chain = map_chain | synthesis_prompt | llm | StrOutputParser()

# ‐‐‐ 运行链 ‐‐‐
async def run_parallel_example(topic: str) -> None:
    """
    异步调用并行处理链,输出综合结果。
    Args:
    topic: 传递给 LangChain 的主题输入
    """
    print(f"\n--- 并行 LangChain 示例,主题:'{topic}' ---")
    try:
        # `ainvoke` 的输入是单个 topic 字符串,
        # 会传递给 map_chain 中的每个 runnable
        response = await full_parallel_chain.ainvoke(topic)
        print("\n- 最终响应 -")
        print(response)
    except Exception as e:
        print(f"\n 链执行出错:{e}")

if __name__ == "__main__":

    test_topic = "太空探索的历史"

    await run_parallel_example(test_topic)


--- 并行 LangChain 示例,主题:'太空探索的历史' ---

- 最终响应 -
太空探索的历史是人类不断突破技术极限、拓展认知边界的过程。从20世纪中叶至今，这一历程不仅见证了科学技术的飞跃，也体现了国际合作与商业力量的演变。以下是基于关键里程碑的梳理，并结合相关问题的延伸分析：

---

### **一、主要历史里程碑**
1. **1957年**：苏联发射首颗人造卫星 **“斯普特尼克1号”**，标志着太空时代的开启，引发美苏太空竞赛。
2. **1961年**：苏联宇航员 **尤里·加加林** 乘坐“东方1号”进入太空，成为首位进入太空的人类。
3. **1969年**：美国 **“阿波罗11号”** 任务成功，宇航员尼尔·阿姆斯特朗与巴兹·奥尔德林实现首次载人登月。
4. **1970年代**：  
   - 空间站兴起：苏联“礼炮号”系列与美国“天空实验室”奠定长期太空居住基础。  
   - 深空探测：探测器如“旅行者”“海盗号”探索火星及外行星。
5. **1981年**：美国 **航天飞机** 首飞，实现可重复使用航天器，降低近地轨道任务成本。
6. **1998年**： **国际空间站（ISS）** 开始建设，成为多国合作的典范，持续运行至今。
7. **21世纪以来**：  
   - 私营企业崛起：SpaceX、蓝色起源等推动商业航天，实现火箭回收、商业载人飞行。  
   - 深空探索：探测器如“好奇号”“毅力号”探索火星，“新视野号”飞掠冥王星。
8. **未来方向**：重返月球（如美国“阿尔忒弥斯计划”）、载人火星任务、深空探测及太空资源利用。

---

### **二、相关问题延伸解析**
#### **1. 阿波罗计划前的关键技术与尝试**
- **火箭技术突破**：  
  - 苏联“R-7”火箭奠定基础，美国“土星五号”重型火箭是关键，其推力足以将飞船送往月球。  
  - 早期试验包括“水星计划”（验证载人飞行）和“双子座计划”（测试太空对接、舱外活动）。
- **无人探测器探路**：  
  - 美国“徘徊者”“勘测者”系列拍摄月球表面，验证软着陆技术；  
  - 苏联“月球号”探测器实现首次月面软着陆并传回数据。
- **生命保障与导航系统**：  
  - 开发密闭舱环境控制、宇航服防护及高精度轨道计

In [5]:
import os
import asyncio
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable, RunnableParallel, RunnablePassthrough

llm = ChatOpenAI(
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("BASE_URL"),
    model="deepseek-chat",
    temperature=0,
)

# ‐‐‐ 定义独立链 ‐‐‐
# 三个链分别执行不同任务,可并行运行
summarize_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "请简明扼要地总结以下主题:"),
            ("user", "{topic}"),
        ]
    )
    | llm
    | StrOutputParser()
)

question_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "请针对以下主题生成三个有趣的问题:"),
            ("user", "{topic}"),
        ]
    )
    | llm
    | StrOutputParser()
)

terms_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "请从以下主题中提取 5-10 个关键词,用逗号分隔:"),
            ("user", "{topic}"),
        ]
    )
    | llm
    | StrOutputParser()
)

# ‐‐‐ 构建并行 + 汇总链 ‐‐‐

# 1. 定义并行任务块,结果与原始 topic 一起传递到下一步
map_chain = RunnableParallel(
    {
        "summary": summarize_chain,
        "questions": question_chain,
        "key_terms": terms_chain,
        "topic": RunnablePassthrough(),  # 传递原始 topic
    }
)

# 2. 定义最终汇总 prompt,整合并行结果
synthesis_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """根据以下信息:
            摘要:{summary}
            相关问题:{questions}
            关键词:{key_terms}
            请综合生成完整答案。""",
        ),
        ("user", "原始主题:{topic}"),
    ]
)

# 3. 构建完整链,将并行结果直接传递给汇总 prompt,再由 LLM 和输出解析器处理
full_parallel_chain = map_chain | synthesis_prompt | llm | StrOutputParser()

# ‐‐‐ 运行链 ‐‐‐
async def run_parallel_example(topic: str) -> None:
    """
    异步调用并行处理链,输出综合结果。
    Args:
    topic: 传递给 LangChain 的主题输入
    """
    print(f"\n--- 并行 LangChain 示例,主题:'{topic}' ---")
    try:
        # 步骤1: 执行并行任务，获取中间结果
        print("\n[步骤1] 开始并行执行三个任务...")
        parallel_results = await map_chain.ainvoke(topic)
        
        # 显示中间结果
        print("\n[中间结果] 并行任务执行完成:")
        print(f"\n  📝 摘要 (summary):")
        print(f"  {parallel_results['summary']}")
        print(f"\n  ❓ 问题 (questions):")
        print(f"  {parallel_results['questions']}")
        print(f"\n  🔑 关键词 (key_terms):")
        print(f"  {parallel_results['key_terms']}")
        print(f"\n  📌 原始主题 (topic):")
        print(f"  {parallel_results['topic']}")
        
        # 步骤2: 执行汇总步骤
        print("\n[步骤2] 开始汇总整合...")
        synthesis_result = await (synthesis_prompt | llm | StrOutputParser()).ainvoke(parallel_results)
        
        print("\n[最终响应]")
        print(synthesis_result)
        
    except Exception as e:
        print(f"\n❌ 链执行出错: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":

    test_topic = "太空探索的历史"

    await run_parallel_example(test_topic)


--- 并行 LangChain 示例,主题:'太空探索的历史' ---

[步骤1] 开始并行执行三个任务...

[中间结果] 并行任务执行完成:

  📝 摘要 (summary):
  太空探索始于20世纪中叶，主要里程碑包括：

1. **1957年**：苏联发射首颗人造卫星“斯普特尼克1号”，开启太空时代。
2. **1961年**：苏联宇航员加加林成为首位进入太空的人类。
3. **1969年**：美国“阿波罗11号”任务实现首次载人登月。
4. **1970年代**：空间站兴起（如礼炮号、天空实验室），深空探测器探索行星。
5. **1981年**：美国航天飞机首飞，实现可重复使用航天器。
6. **1998年**：国际空间站开始建设，开启多国长期合作。
7. **21世纪**：私营企业加入（如SpaceX），推动商业航天；探测器探索火星、外行星及太阳系边缘。
8. **未来目标**：重返月球、载人火星任务及深空探索。

太空探索推动了科技、通信和地球科学研究，成为人类探索未知的重要领域。

  ❓ 问题 (questions):
  1. 在阿波罗计划之前，人类为登月做了哪些关键的技术突破和尝试？  
2. 国际空间站的建设过程中，不同国家之间如何协调技术标准与合作分工？  
3. 未来商业太空旅行可能如何改变普通人对宇宙的认知和生活方式？

  🔑 关键词 (key_terms):
  太空探索,历史,航天,宇航员,火箭,卫星,登月,国际空间站,探测器,航天飞机

  📌 原始主题 (topic):
  太空探索的历史

[步骤2] 开始汇总整合...

[最终响应]
太空探索是人类探索未知、拓展生存边界的重要历程，其历史跨越了从20世纪中叶至今的多个关键阶段。以下是结合**太空探索的历史**主线，对相关技术与里程碑的梳理，并针对具体问题的延伸分析：

---

### **一、太空探索的历史主线**
1. **起步阶段（1950-1960年代）**  
   - **1957年**：苏联发射首颗人造卫星“斯普特尼克1号”，标志着太空时代的开启。  
   - **1961年**：苏联宇航员**加加林**乘坐“东方1号”首次进入太空，实现载人航天突破。  
   - **1969年**：美国“阿波罗11号”任务成功登月，**阿姆斯特朗**成为首

In [7]:
import os

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, Runnable


llm = ChatOpenAI(
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("BASE_URL"),
    model="deepseek-chat",
    temperature=0,
)



# ---------------- 1. 三个“调研员”子链（相当于 LlmAgent） ----------------

# 调研员 1：可再生能源
renewable_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一名专注于能源领域的 AI 调研助手。",
        ),
        (
            "user",
            "请基于你掌握的公开信息，调研“可再生能源最新进展”，"
            "用 1-2 句中文简明总结关键发现，只输出摘要。",
        ),
    ]
)
renewable_chain: Runnable = renewable_prompt | llm | StrOutputParser()

# 调研员 2：电动汽车
ev_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一名专注于交通领域的 AI 调研助手。",
        ),
        (
            "user",
            "请基于你掌握的公开信息，调研“电动汽车技术最新进展”，"
            "用 1-2 句中文简明总结关键发现，只输出摘要。",
        ),
    ]
)
ev_chain: Runnable = ev_prompt | llm | StrOutputParser()

# 调研员 3：碳捕集
carbon_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一名专注于气候解决方案的 AI 调研助手。",
        ),
        (
            "user",
            "请基于你掌握的公开信息，调研“碳捕集方法现状”，"
            "用 1-2 句中文简明总结关键发现，只输出摘要。",
        ),
    ]
)
carbon_chain: Runnable = carbon_prompt | llm | StrOutputParser()


# ---------------- 2. 并行调研（相当于 ParallelAgent） ----------------
# RunnableParallel 会并行调用内部每条 runnable，
# 返回一个 dict，key 就是你这里起的名字。

parallel_research_chain: Runnable = RunnableParallel(
    {
        "renewable_energy_result": renewable_chain,
        "ev_technology_result": ev_chain,
        "carbon_capture_result": carbon_chain,
    }
)


# ---------------- 3. 合并智能体（相当于 SynthesisAgent） ----------------

merge_prompt = ChatPromptTemplate.from_template(
    """
你是一名负责整合调研结果的 AI 助手。
你的任务是将以下调研摘要合成为结构化报告，并明确归属。
每个主题用标题分段，确保内容连贯，仅整合输入摘要。

注意：你的全部回答必须严格基于下方“输入摘要”，
不得添加任何外部知识或细节。

**输入摘要:**

**可再生能源:**
{renewable_energy_result}

**电动汽车:**
{ev_technology_result}

**碳捕集:**
{carbon_capture_result}

**输出格式:**

## 可持续技术最新进展摘要

### 可再生能源发现
(基于 RenewableEnergyResearcher 的摘要，仅整合上述内容)

### 电动汽车发现
(基于 EVResearcher 的摘要，仅整合上述内容)

### 碳捕集发现
(基于 CarbonCaptureResearcher 的摘要，仅整合上述内容)

### 总结
(仅基于上述内容，简要总结 1-2 句)

只输出结构化报告，严格按上述格式，不加其他说明。
"""
)

merge_chain: Runnable = merge_prompt | llm | StrOutputParser()


# ---------------- 4. 串行“总控智能体”（相当于 SequentialAgent） ----------------
# 这里用 LCEL 的链式写法：先并行调研，再合并。
# 注意这行的写法就和你之前看到的
# question_chain: Runnable = ( ... )
# 是一个套路：右边整个表达式被括号包起来，可以多行换行。

pipeline: Runnable = (
    parallel_research_chain  # -> dict: {renewable_energy_result, ...}
    | merge_chain            # 接上面的 dict 作为 merge_prompt 的输入
)


if __name__ == "__main__":
    # 因为我们没用任何输入变量，随便传个 {} 即可
    report = pipeline.invoke({})
    print(report)


## 可持续技术最新进展摘要

### 可再生能源发现
(基于 RenewableEnergyResearcher 的摘要，仅整合上述内容)
全球可再生能源装机持续高速增长，光伏与风电成本竞争力显著增强。新型储能技术及绿氢制备成为其规模化应用的关键突破方向。

### 电动汽车发现
(基于 EVResearcher 的摘要，仅整合上述内容)
固态电池技术加速落地，800V高压平台与碳化硅器件的普及正推动超快充普及。智能驾驶技术正向城市NOA与舱驾融合方向深化发展。

### 碳捕集发现
(基于 CarbonCaptureResearcher 的摘要，仅整合上述内容)
目前碳捕集技术以燃烧后捕集为主流，其中化学吸收法相对成熟但能耗与成本较高。燃烧前捕集和富氧燃烧技术处于示范阶段，而新兴的直接空气捕集技术仍面临高成本挑战，整体产业化需进一步降本增效。

### 总结
(仅基于上述内容，简要总结 1-2 句)
可再生能源、电动汽车和碳捕集技术均在快速发展，其共同的关键挑战与突破方向均聚焦于降低成本、提升效率以实现规模化应用。
